In [24]:
import facebook
import json
import requests
import uuid

# Setting the access token to use the facebook api 

In [203]:
ACCESS_TOKEN='EAACEdEose0cBALpApC3y6GwYxn92f2lMGr2VJen2LXaABjgCfVP1wpXA8nq8gjDTBudOlWGlvX2ai6qctZCDtwJ3ZBPE82fnAZAlzHvL1JFJkbcaVgbPCZCu2apVEHUzjL5qIUSeEZAm7u3ShWNqo3YTKwllJP6b9CG6RP2ptilboankAb7Ao4Y9p65czyLdjYZD'

In [204]:
g = facebook.GraphAPI(ACCESS_TOKEN)

# Verifying the facebook api 

In [ ]:
g.get_object('me')

In [194]:
def pp(o):
    print json.dumps(o, indent=1)

# Setting the Facebook Page Id e.g. BBC India  

In [211]:
page_id= 'bbcindia'

# retrieve the BBCINDIA Facebook Page Posts

In [33]:
page = g.get_object(page_id+'/posts?fields=message', page=True, retry=5)

# Providing the number of pages to search search

In [212]:
page_count = 1
pages_to_search = 250


# We will be firstly getting the facebook posts and then their related comments


# Function to get the posts


In [213]:
def retrieveFacebookPagePosts(pages_to_search):
    posts_data = []
    while page_count < pages_to_search:
        try:
            for message in page['data']:
                if 'message' in message:
                    post_data = {}
                    commentdata = {}
                    post_data['text'] = message['message']
                    post_data['id'] = str(uuid.uuid4())
                    commentdata['id'] = message['id']
                    post_data['comments'] = commentdata
                    post_data['time'] = message['created_time']
                    posts_data.append(post_data)
                    
            page_count = page_count + 1
            #page = requests.get(page['paging']['next']).json()
            if 'paging' in page and 'next' in page['paging']:
                page = requests.get(page['paging']['next']).json()
            else:
                print "Page count reached "+str(page_count)
                print 'going to break'
                break
        except KeyError:
            print "Exception occured"
    return posts_data


# Function to retrieve the comments of posts

In [214]:
def getPostComments(comment_id):
    comments = []
    comment_data = g.get_object(comment_id)
    comment_page_count = 1
    
    try:
        while comment_page_count < 2 :
            if 'comments' in comment_data:
                for comment in comment_data['comments']['data']:
                    comments.append(comment['message'])

            comment_page_count = comment_page_count + 1
            if 'comments' in comment_data and 'paging' in comment_data['comments'] and 'next' in comment_data['comments']['paging']:
                comment_data = requests.get(comment_data['comments']['paging']['next']).json()
            else:
                break
    except:
        print "exception occured"
        print comment_page_count
        
    return comments

# Fetching the Posts


In [161]:
posts_data = retrieveFacebookPagePosts(pages_to_search)

Page count reached 119
going to break


In [ ]:
import os
os.system('say "your program has finished"')

# Inserting the Comments into the posts

In [179]:
for post in posts_data:
    post['comments']['data'] = getPostComments(post['comments']['id'])

{
 "text": "Disney is calling it the first ever \"exclusively gay moment\" on film.", 
 "id": "125c50ca-3da8-4c1e-aac2-acdf61120a86", 
 "comments": {
  "data": [], 
  "id": "151955124848859_1442089169168775"
 }, 
 "time": "2017-03-02T10:15:00+0000"
}
{
 "text": "Sleeping too much or even too little can be very harmful.", 
 "id": "022e62d8-2ce7-44e8-b375-da7f2f44186a", 
 "comments": {
  "data": [
   "Nisha Mehul Teli"
  ], 
  "id": "151955124848859_1442087229168969"
 }, 
 "time": "2017-03-02T09:10:00+0000"
}


In [217]:
import pickle

fw = open('fb_post_data','wb')
pickle.dump(posts_data,fw)

fr = open('fb_post_data','rb')
pickled_posts = pickle.load(fr)

print len(pickled_posts)

print posts_data == pickled_posts

2877
True


# Keeping only English comments

In [265]:
import langdetect

In [289]:
def retriveEnglistCommentsOnly(comments):
    english_comments = []
    for comment in comments:
        try:
            if langdetect.detect(comment) == 'en':
                english_comments.append(comment+'.')
        except:
            english_comments.append(comment+'.')
    return english_comments

# Text Processing Before inserting into the mongodb

In [290]:
%timeit
#adding text key which will contains the whole post and comment data
for post in posts_data:
    post['text_comment'] = post['text']+u' '+ u' '.join(retriveEnglistCommentsOnly(post['comments']['data']))

In [505]:
posts_data[19]['text_comment']

u"Ms Conway is seen kneeling on the sofa and clutching her phone as US President Donald Trump poses with leaders of historically black colleges and universities. Belonging to The regime which came in and started demeaning the federal offices and system in itself, this isn't a big surprise.... Everybody should keep their dirty feet off the furniture.  Ugh.. Did you grow up in a barn?.. People have to sit there.. This  is being shared to me aprox 20 time daily I do not like it & I will not give it a like. I thought white house had an official photographer.\U0001f609. It is disgusting. No respect for 'White House'.. Pinned her up alongside Melania."

In [294]:
from spacy.en import English
import codecs
import re
import os
import unidecode

In [295]:
parser = English()

In [296]:
fbpost_filepath = os.path.join('fbpagepost_all.txt')

In [ ]:
with codecs.open(fbpost_filepath, 'w', encoding='utf_8') as f:
        for fbpost in posts_data:
                
                post = unidecode.unidecode(fbpost['text_comment'])
                print "Orig: "+post
                #removing out the Repost word, @abc, url and hash from hashtag
                post = re.sub(r"http\S+", " ", post,flags=re.IGNORECASE)
                post = re.sub(r"#", " ", post,flags=re.IGNORECASE)#|RT|@\S+|-|'|%|\'|\(|\)|;|:|&nbsp;|'s|’|\"|&amp|’s|\U000\S+
                post = re.sub(r"RT", " ", post,flags=re.IGNORECASE)
                post = re.sub(r"@\S+", " ", post,flags=re.IGNORECASE)
                #for removing the &gt;
                post = re.sub(r"&\S+", " ", post,flags=re.IGNORECASE)
                post = re.sub(r"-", "",post)
                post = re.sub(r"&nbsp;", " ", post,flags=re.IGNORECASE)
                post = re.sub(r"%", " ",post)
                post = re.sub(r"'s", " ", post,flags=re.IGNORECASE)
                post = re.sub(r"\'", "",post)
                post = re.sub(r"\\", "",post)
                post = re.sub(r"/", "",post)
                post = re.sub(r"\(", " ",post)
                post = re.sub(r"\)", " ",post)
                post = re.sub(r";", " ",post)
                post = re.sub(r":", " ",post)
                post = re.sub(r"[.]{2,}"," ", post)
                post = re.sub(r"\"", " ",post)
                post = re.sub(r"<", " ",post)
                post = re.sub(r">", " ",post)
                post = re.sub(r"\[", " ",post)
                post = re.sub(r"\]", " ",post)
                post = re.sub(r"\{", " ",post)
                post = re.sub(r"\}", " ",post)
                post = re.sub(r"\|", " ",post)
                post = re.sub(r"=", " ",post)
                post = re.sub(r"~", " ",post)
                post = re.sub(r"\`", " ",post)
                post = re.sub(r"\^", " ",post)
                post = re.sub(r"\+", " ",post)
                post = re.sub(r"!", " ",post)
                post = re.sub(r"\*", " ",post)
                
                post = re.sub(r" [a-z0-9] ", " ",post)
               # post = re.sub(r"^[a-z0-9] ", " ",post)
               # post = re.sub(r" [a-z0-9]$", " ",post)
                #post = re.sub(r"[a-z]", "",post)
                post = re.sub(r"&amp", " ", post,flags=re.IGNORECASE)
                post = re.sub(r"\\U\S+", " ", post,flags=re.IGNORECASE)
                post = " ".join(post.split())
                print "Axed: "+post
                f.write(post+'\n')

In [302]:
def punct_space_stopword(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space or token.is_stop

def post_line(filename):
    """
    generator function to read in post from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for post in f:
            yield post.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse post,
    lemmatize the text, and yield sentences
    """
    
    for parsed_post in parser.pipe(post_line(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_post.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space_stopword(token)])

In [303]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
import pandas as pd
import itertools as it

In [310]:
unigram_sentences_filepath = os.path.join('fbpost_unigram_sentences_all.txt')

In [311]:
%%time
with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for sentence in lemmatized_sentence_corpus(fbpost_filepath):
            f.write(sentence + '\n')

CPU times: user 23.6 s, sys: 248 ms, total: 23.8 s
Wall time: 24.4 s


In [312]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

In [313]:
for unigram_sentence in it.islice(unigram_sentences, 200, 210):
    print u' '.join(unigram_sentence)
    print u''

man tell open iranian people .this man adam purinton 51 flee scene attack pub olathe go restaurant confide kill iranian immigrant

thing indians bear close affinity iranians founder islamic terrorism

indians maintain indianism tilak wear forehead easily identify hope treat foreign land

trump plicy annihilation islamic terrorism swips wide scale acclamation world

question security indian diospora

view distinguish self india culture manner ward evil

muslim fanatics bhakts turn india rigid country like saudi arabia

india secularism root hinduism fundamentally open religion school thought

core ideology hinduism describe upanishad vasudhaiva kutumbakam world family btw iranians zoroastrian force conversion islam

victim desi muslims



In [314]:
bigram_model_filepath = os.path.join('fbpost_bigram_model_all')

In [315]:
bigram_model = Phrases(unigram_sentences)
bigram_model.save(bigram_model_filepath)

#loading the model
bigram_model = Phrases.load(bigram_model_filepath)

In [316]:
bigram_sentences_filepath = os.path.join('fbpost_bigram_sentences_all.txt')

In [317]:
%%time
with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for unigram_sentence in unigram_sentences:    
        bigram_sentence = u' '.join(bigram_model[unigram_sentence])
        f.write(bigram_sentence + '\n')

CPU times: user 1.12 s, sys: 77.7 ms, total: 1.2 s
Wall time: 1.27 s


/Users/amantandon/anaconda/lib/python2.7/site-packages/gensim/models/phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [318]:
bigram_sentences = LineSentence(bigram_sentences_filepath)
for bigram_sentence in it.islice(bigram_sentences, 2020, 2150):
    print u' '.join(bigram_sentence)
    print u''

undo achieve talk theresa donald_trump say petition get 900,000 signature react muslim country strict trump

people potus visit uk mainly work intent

extensive method brexit calculations

boe understand people intention

go wrong liverpool fc

official ningbo zoo shoot tiger dead attack man enter enclosure shoot person enter enclosed area sound hea f

not feel pity idiot r.i.p.

tiger

wish not jump area

alive poor animal pay stupidity mankind idiot b4 shoot tiger compare population tiger population human china

murder poor tiger fault shame zoo authorities

tiger bite dust coz lunatic decide hello agree enclosure animal personal space

intruder shoot occupant bc tranquilizer gun hear .then k run zoo shoot tiger

fault man enter territory

hv shoot man enter zoo shame thm

shoot man instead idiots shoot man

animal die stupidty man

can_not believe people not hv mercy animala

stupidity height

let tiger home

bbc china dear

china

intense

widely condemn

ban ce_ainly save america 

In [319]:
trigram_model_filepath = os.path.join('fbpost_trigram_model_all')

In [320]:
trigram_model = Phrases(bigram_sentences)
trigram_model.save(trigram_model_filepath)

In [321]:
trigram_model = Phrases.load(trigram_model_filepath)

In [322]:
trigram_sentence_filepath = os.path.join('fbpost_trigram_sentences.txt')

In [323]:
%%time
with codecs.open(trigram_sentence_filepath, 'w', encoding='utf-8') as f:
    for sentence in bigram_sentences:
        trigram_sentence = u' '.join(trigram_model[sentence])
        f.write(trigram_sentence+'\n')

CPU times: user 1.07 s, sys: 76.5 ms, total: 1.15 s
Wall time: 1.2 s


In [324]:
trigram_sentences = LineSentence(trigram_sentence_filepath)

In [329]:
for unigram_sentence in it.islice(trigram_sentences, 2020, 2150):
    print u' '.join(unigram_sentence)
    print u''

undo achieve talk theresa donald_trump say petition get 900,000 signature react muslim country strict trump

people potus visit uk mainly work intent

extensive method brexit calculations

boe understand people intention

go wrong liverpool fc

official ningbo zoo shoot tiger dead attack man enter enclosure shoot person enter enclosed area sound hea f

not feel pity idiot r.i.p.

tiger

wish not jump area

alive poor animal pay stupidity mankind idiot b4 shoot tiger compare population tiger population human china

murder poor tiger fault shame zoo authorities

tiger bite dust coz lunatic decide hello agree enclosure animal personal space

intruder shoot occupant bc tranquilizer gun hear .then k run zoo shoot tiger

fault man enter territory

hv shoot man enter zoo shame thm

shoot man instead idiots shoot man

animal die stupidty man

can_not_believe people not hv mercy animala

stupidity height

let tiger home

bbc china dear

china

intense

widely condemn

ban ce_ainly save america 

In [ ]:
for parsed_post in parser.pipe(post_line(bigram_sentences_filepath),
                                  batch_size=10000, n_threads=4):
        
        for num, entity in enumerate(parsed_post.ents):
            print 'Entity {}:'.format(num + 1), entity, '-', entity.label_
            print ''

# trying to find out the topics: Topic Modelling

In [413]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore
import gensim

import pyLDAvis
import pyLDAvis.gensim
import warnings
import cPickle as pickle

In [414]:
#creating the dictionary
trigram_dictionary_filepath = os.path.join('trigram_dict_all.dict')

In [454]:
%%time


trigram_posts = LineSentence(trigram_sentence_filepath)

# learn the dictionary by iterating over all of the post
trigram_dictionary = Dictionary(trigram_posts)
    
# filter tokens that are very rare or too common from
# the dictionary (filter_extremes) and reassign integer ids (compactify)
trigram_dictionary.filter_extremes(no_below=20, no_above=0.4)
trigram_dictionary.compactify()

trigram_dictionary.save(trigram_dictionary_filepath)
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

CPU times: user 581 ms, sys: 25 ms, total: 606 ms
Wall time: 624 ms


In [457]:
trigram_bow_filepath = os.path.join('trigram_bow_corpus_all.mm')

In [458]:
def trigram_bow_generator(filepath):
    """
    generator function to read post from a file
    and yield a bag-of-words representation
    """
    
    for post in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(post)

In [459]:
%%time
# generate bag-of-words representations for
# all post and save them as a matrix
MmCorpus.serialize(trigram_bow_filepath,
                       trigram_bow_generator(trigram_sentence_filepath))
    
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

CPU times: user 920 ms, sys: 83.2 ms, total: 1 s
Wall time: 1.11 s


In [460]:
lda_model_filepath = os.path.join('lda_model_all')

In [461]:
%%time


with warnings.catch_warnings():
    warnings.simplefilter('ignore')
        
    # workers => sets the parallelism, and should be
    # set to your number of physical cores minus one
    lda = LdaMulticore(trigram_bow_corpus,
                           num_topics=20,
                           id2word=trigram_dictionary,
                           workers=3)
    
lda.save(lda_model_filepath)
    
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

CPU times: user 4.16 s, sys: 2.68 s, total: 6.85 s
Wall time: 7.48 s


In [462]:
lda.print_topics(-1)

[(0,
  u'0.031*"india" + 0.020*"business" + 0.017*"bbc" + 0.016*"indian" + 0.013*"people" + 0.011*"lot" + 0.011*"get" + 0.010*"phone" + 0.008*"brand" + 0.008*"not"'),
 (1,
  u'0.020*"happen" + 0.017*"great" + 0.013*"bbc" + 0.013*"bbc_travel" + 0.012*"india" + 0.009*"mobile_phone" + 0.009*"world" + 0.009*"people" + 0.009*"repo" + 0.009*"fast"'),
 (2,
  u'0.031*"india" + 0.029*"people" + 0.022*"work" + 0.020*"car" + 0.017*"indian" + 0.013*"stop" + 0.013*"not" + 0.012*"bbc" + 0.010*"need" + 0.010*"country"'),
 (3,
  u'0.032*"india" + 0.030*"people" + 0.016*"live" + 0.014*"like" + 0.013*"woman" + 0.013*"channel" + 0.011*"hi" + 0.010*"not" + 0.010*"wonder" + 0.009*"good"'),
 (4,
  u'0.059*"india" + 0.019*"woman" + 0.014*"best" + 0.013*"man" + 0.012*"world" + 0.010*"air" + 0.009*"girl" + 0.008*"kashmir" + 0.008*"oh" + 0.008*"point"'),
 (5,
  u'0.029*"time" + 0.022*"look" + 0.021*"india" + 0.020*"gear" + 0.018*"love" + 0.013*"hi" + 0.013*"good" + 0.013*"device" + 0.011*"suppo" + 0.011*"guy"')

In [463]:
tfidf_model = gensim.models.TfidfModel(trigram_bow_corpus, id2word=trigram_dictionary)

In [484]:
%%time
lsi_model = gensim.models.LsiModel(tfidf_model[trigram_bow_corpus], id2word=trigram_dictionary, num_topics=3000)

CPU times: user 33.1 s, sys: 1.94 s, total: 35.1 s
Wall time: 19.5 s


In [485]:
def explore_topic_lsi(topic_number, topn=25):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print u'{:20} {}'.format(u'term', u'frequency') + u'\n'

    for term, frequency in lsi_model.show_topic(topic_number, topn=25):
        print u'{:20} {:.3f}'.format(term, round(frequency, 3))

In [486]:
def explore_topic(topic_number, topn=25):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print u'{:20} {}'.format(u'term', u'frequency') + u'\n'

    for term, frequency in lda.show_topic(topic_number, topn=25):
        print u'{:20} {:.3f}'.format(term, round(frequency, 3))

In [487]:
explore_topic(topic_number=19)

term                 frequency

india                0.090
grow                 0.018
bbc                  0.015
british              0.015
season               0.014
take                 0.010
sta                  0.009
help                 0.009
world                0.009
place                0.008
like                 0.008
discover             0.007
today                0.007
play                 0.007
channel              0.006
delhi                0.006
say                  0.006
news                 0.006
thing                0.005
work                 0.005
dr                   0.005
pa                   0.005
human                0.005
late                 0.005
performance          0.005


In [488]:
explore_topic_lsi(topic_number=100)

term                 frequency

ask                  -0.329
police               -0.322
bjp                  0.305
win                  0.211
picture              0.198
hindu                0.189
save                 -0.188
lot                  -0.176
question             -0.169
guy                  -0.169
animal               -0.167
hea                  0.162
home                 0.150
respect              -0.126
poor                 0.125
delhi                -0.123
god_bless            0.120
water                -0.110
miss                 0.104
believe              0.102
word                 0.101
suppo                -0.093
kashmir              -0.088
ban                  -0.087
job                  -0.087


In [500]:
type(explore_topic_lsi(topic_number=300))#700

term                 frequency

hold                 0.232
destroy              0.199
temple               0.181
political            0.180
tiger                0.173
shoot                -0.166
crime                0.161
step                 0.148
village              0.139
democracy            -0.131
fool                 -0.128
white                -0.128
student              -0.124
wear                 0.123
enjoy                -0.122
level                0.119
away                 0.115
sign                 -0.113
arrest               -0.112
product              -0.111
set                  0.107
number               -0.104
action               -0.102
president            -0.102
example              0.100


NoneType

In [502]:
type(lsi_model.show_topic(300))

list